## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup darknet environment

In [ ]:
!nvidia-smi

In [ ]:
import os
import cv2
cv2.__version__

%cd /content
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# remove the existing folder
!rm -r darknet_for_colab

# download and compile darknet_for_colab
!git clone https://github.com/quangnhat185/darknet_for_colab.git
%cd darknet_for_colab

!sed -i '21d' /content/darknet_for_colab/Makefile  
!sed -i '21d' /content/darknet_for_colab/Makefile  
!sed -i '21d' /content/darknet_for_colab/Makefile  
!sed -i '21d' /content/darknet_for_colab/Makefile  
!sed -i '20 s@-gencode arch=compute_30,code=sm_30\ @-gencode arch=compute_70,code=[sm_70,compute_70]@' /content/darknet_for_colab/Makefile #V100
# !sed -i '20 s@-gencode arch=compute_30,code=sm_30\ @-gencode arch=compute_60,code=sm_60@' /content/darknet_for_colab/Makefile #P100
!sed -i '3 s@CUDNN_HALF=0@CUDNN_HALF=1@' /content/darknet_for_colab/Makefile 
!sed -i '5 s@AVX=0@AVX=1@' /content/darknet_for_colab/Makefile 
!sed -i '6 s@OPENMP=0@OPENMP=1@' /content/darknet_for_colab/Makefile  

!make
!chmod +x ./darknet

## Download yolov4 pre-trained weights

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

## Download and visualise dataset

In [ ]:
!rm /content/darknet_for_colab/data -r
!mkdir /content/darknet_for_colab/data
%cd /content/darknet_for_colab/data
!unzip /content/drive/MyDrive/YOLOv4_weight/yolo_monologo_labelled.zip 

%cd /content/darknet_for_colab

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob

def read_label(image_path):
  file_name = image_path.replace('.jpg', '.txt')
  with open(file_name, 'rt') as file:
    print(os.path.basename(file_name) + ': \n' + file.read())

image_path = glob.glob("data/ts/*.jpg")
fig = plt.figure(figsize=(12,8))
cols = 2
rows = 2
grid = gridspec.GridSpec(nrows=rows, ncols=cols, figure=fig)
for i in range(cols*rows):
  fig.add_subplot(grid[i])
  image=plt.imread(image_path[i])
  plt.title(os.path.basename(image_path[i]))
  plt.axis(False)
  plt.imshow(image)
  read_label(image_path[i])

plt.savefig("dataset_examples.jpg", dpi=300)

## Modify yolov4 architecture

 

In [ ]:
num_classes = 5
num_batches = 5000 
width = 1280
height = 704
num_subdiv = 64

!sed -i '14 s@classes=4@classes={num_classes}@' /content/darknet_for_colab/yolov4_config.py
!sed -i '15 s@max_batches=8000@max_batches={num_batches}@' /content/darknet_for_colab/yolov4_config.py
!sed -i '17 s@subdivisions=16@subdivisions={num_subdiv}@' /content/darknet_for_colab/yolov4_config.py
!sed -i '18 s@width=416@width={width}@' /content/darknet_for_colab/yolov4_config.py
!sed -i '19 s@height=416@height={height}@' /content/darknet_for_colab/yolov4_config.py

In [ ]:
%cd /content/darknet_for_colab 
!python yolov4_setup.py

In [ ]:
!rm /content/darknet_for_colab/backup -r
!ln -s /content/drive/'MyDrive'/YOLOv4_weight/backup /content/darknet_for_colab

## Training

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
import matplotlib.pyplot as plt
os.chdir('/content/darknet_for_colab/')
!chmod +x darknet
!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg yolov4.conv.137 -points 0 -map -dont_show 2> /dev/null
fig = plt.figure(figsize=(10,10))
train_result = plt.imread("chart.png")
plt.axis(False)
plt.imshow(train_result)
plt.savefig('training_fig.png')

In [ ]:
!./darknet detector map data/yolov4.data cfg/yolov4_custom_test.cfg /content/drive/MyDrive/YOLOv4_weight/backup/yolov4_custom_train_best.weights

## Test with example image/video

In [ ]:
%cd /content/darknet_for_colab
%cp data/yolov4.data cfg/coco.data

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
img_path = "data/ts/frame1099.jpg"
!./darknet detect cfg/yolov4_custom_test.cfg /content/drive/MyDrive/YOLOv4_weight/backup/yolov4_custom_train_best.weights {img_path} -dont-show  

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(12,12))
plt.axis(False)
processed_image = plt.imread("./predictions.jpg")
plt.imshow(processed_image)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
# video processed
!python darknet_video.py -v /content/test_vid.mp4 -c cfg/yolov4_custom_test.cfg -w /content/drive/MyDrive/YOLOv4_weight/backup/yolov4_custom_train_best.weights -o output.mp4